<a href="https://colab.research.google.com/github/Uriel825/Experimentos-Pytorch/blob/master/RedesCycleGan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive')


In [0]:
%cd '/gdrive/My Drive'

/gdrive/My Drive


In [0]:
!git clone https://github.com/aitorzip/PyTorch-CycleGAN.git

fatal: destination path 'PyTorch-CycleGAN' already exists and is not an empty directory.


In [0]:
%cd '/gdrive/My Drive/PyTorch-CycleGAN'

/gdrive/My Drive/PyTorch-CycleGAN


In [0]:
#%%sh
#sh ./download_dataset summer2winter_yosemite

In [0]:
!mv datasets/summer2winter_yosemite /gdrive/My\ Drive/dl-pytorch/datasets/


mv: cannot stat 'datasets/summer2winter_yosemite': No such file or directory


In [0]:
!ls /gdrive/My\ Drive/dl-pytorch/datasets/

64x64_SIGNS	 cifar-10-batches-py	 summer2winter_yosemite
64x64_SIGNS.zip  cifar-10-python.tar.gz


In [0]:
import torch.nn as nn
import torch.nn.functional as F



In [0]:
class ResidualBlock(nn.Module):
  def __init__(self, in_features):
    super(ResidualBlock, self).__init__()

    conv_block = [ nn.ReflectionPad2d(1), #rellena la red
                 nn.Conv2d(in_features, in_features, 3),
                 nn.InstanceNormd2d(in_features), #BN para gans
                 nn.ReLU(True),
                 nn.ReflectionPad2d(1), #mejor para mantener la distribucion
                 nn.Conv2d(in_features, in_features, 3),
                 nn.InstanceNormd2d(in_features)
                ]  
    self.conv_block = nn.Sequential(*conv_block)
  
  def forward(self, x):
    return self.conv_block(x) + x #Juega con el espacio de modelos 



In [0]:
class Generator(nn.Module):
  def __init__(self, input_nc, output_nc, n_residuals_blocks=9):
    super(Generator,self).__init__()
    
    #Bloque convulusional
    model = [ nn.ReflectionPad2d(3),
              nn.Conv2d(input_nc, 64, F), #I-7+6/1 +1 = I
              nn.InstanceNorm2d (64),
              nn.ReLU(True)
            ]
    in_features = 64
    out_features = in_features * 2
     
    #Capa de enconding
    for _ in range(2):
      model += [ nn.Conv2d(in_features, out_features, 3, stride=2, padding=1), #I/2 
                 nn.InstanceNorm2d(out_features),
                 nn.ReLU(True)
               ]
      in_features = out_features
      out_features = in_features*2
    #Tranformasiones residuales

    for _ in range(n_residuals_blocks):
      model += [ResidualBlock(in_features)]
    
    #Capa de decoding
    out_features = in_features//2
    for _ in range(2):
      model += [nn.ConvTranspose2d(in_features, out_features, 3, stride=2, padding=1, output_padding=1),#I*2
                nn.InstanceNorm2d(out_features),
                nn.ReLU(True)
               ]
      in_features = out_features
      out_features = in_features //2

      #salida
      model += [ nn.ReflectionPad2d(3),
                 nn.Conv2d(64, output_nc, 7),#I
                 nn.Tanh()
              ] 

      self.model = nn.Sequential(*model)

      #Metodo Fordward
      def fordward(self,x):
        return self.model(x)

In [0]:
#Red Discriminativa

class Discrimanator(nn.Module):
  "patchGAN: Discrimina estilo o Textura"
  def __init__(self, input_nc):
    super(Discrimanator, self).__init__()

    model = [ nn.Conv2d(input_nc, 64, 4, stride=2, padding=1), #I/2
              nn.LeakyReLU(0.2, in_place=True)
            ]
    
    model += [ nn.Conv2d(64, 128, 4, stride=2, padding=1), #I/2
               nn.InstanceNorm2d(128),
               nn.LeakyReLU(0.2, in_place=True)
             ]
    model += [ nn.Conv2d(128, 256, 4, stride=2, padding=1), #I/2
               nn.InstanceNorm2d(256),
               nn.LeakyReLU(0.2, in_place=True)
             ]
    
    model += [ nn.Conv2d(256, 512, 4, padding=1), #I/2
               nn.InstanceNorm2d(256),
               nn.LeakyReLU(0.2, in_place=True)
             ]
    
    #Flatten
    model += [nn.Conv2d(512, 1, 4, padding=1)] #I-1

    self.model == nn.Sequential(*model)

  def forward(self, x):
    x = self.model(x)
    return F.avg_pool2d(x, x.size()[2:]).view(x.size()[0], -1)


In [0]:
#Entrenamiento
import sys
import argparse
import glob
import random
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

sys.path.append('/gdrive/My Drive/dl-pytorch')

In [0]:
class ImageDataset(Dataset):
  def __init__(self, base_dir, transform=None, split='train'):
    self.tramsform = transforms.Compose(transform) #Hack pemite quitar el if x asocia a x
    self.file_A = sorted(glob.glob(os.path.join(base_dir,'{}/A/*.*'.format(split)))) #se usa para ir al base dir de A va a ir a buscar las imagenes con extensiones 
    self.file_B = sorted(glob.glob(os.path.join(base_dir,'{}/B/*.*'.format(split)))) #sorted = ordenado
  def __len__ (self):
    return max(len(self.files_a), len(self.files_B))
  def __getitem__(self,idx): #idx = index
    image_A = self.transform(Image.open(self.files_A[idx]))
    image_B = self.transform(Image.open(self.files_A[random.randint(0,len(self.files_B)-1)]))
    return {'A':image_A, 'B':image_B}
      

  Instanciando todo lo necesario





In [0]:
epoch = 0
n_epochs = 200
batch_size = 4 #Es bueno entrenar con smalls batchs
lr = 1e-3
size = 256
input_nc = 3
output_nc = 3
decay_epoch = 100 #pending

cuda=True
n_cpu = 8 #Paralizar los datos

base_dir = '/gdrive/My Drive/dl-pytorch/datasets/summer2winter_yosemite'



In [0]:
device = torch.device('cuda' if cuda else 'cpu')
def weights_init_normal(m):
  classname = m.__class.__name__
  if classname.find('Conv') !=1:
      torch.nn.init.normal(m.weight.data, 0.0, 1e-2) #Inicializacion con distribucion gaussiana
  elif classname.find('BatchNorm2d') !=1:
      torch.nn.init.normal(m.weight.data, 1.0, 1e-2) #solo cambiar el 0 por 1
      torch.nn.init.constant(m.bias, 0.0) #capa bias

netG_A2B = Generator(input_nc, output_nc)
netG_B2A = Generator(output_nc, input_nc)
netD_A = Discriminator(input_nc)
netD_B = Discriminator(output_nc)

if cuda:
      netG_A2B.to(device)
      netG_B2A.to(device)
      netD_A.to(device)
      netD_B.to(device)

netG_A2B.apply(weights_init_normal)
netG_B2A.apply(weights_init_normal)
netD_A.apply(weights_init_normal)
netD_B.apply(weights_init_normal)

# Lossess
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()



Optmizadores y scheduales

In [0]:
optimizer_G = torch.optim.Adam(itertools.chain(netG_A2B.parameters(), netGB2A.parameters()),#Juntar parametros y entrenar al mismo tiempo
                               lr=lr, betas=(0.5, 0.999)) 
optmizer_D_A = torch.optim.Adam(netD_A().parameters, lr=lr, betas=(0.5, 0.999))
optmizer_D_B = torch.optim.Adam(netD_B().parameters, lr=lr, betas=(0.5, 0.999))

#schedualers (Permiten actualizar el Learning rate de manera dinamica durante el entrenamiento)
class LambdaLR():
    def __init__ (self, n_epochs, offset, decay_start_epoch): #El decaimiento del epcoh se hace cuando ya llava altos parametros de entrenamiento
      assert((n_epochs-decay_start_epoch)>0)
      self.n_epochs = n_epochs
      self.offset = offset
      self.decay_start_epoch = decay_start_epoch
    def step(self, epoch):
      return 1 - max(0,epoch + self.offset - self.decay_start_epoch)/(self.n_epochs - self.decay_start_epoch) 

lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=LambdaLR(n_epochs,epoch,decay_epoch).step) #Permite crear un funcion lambda
lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_D_A, lr_lambda=LambdaLR(n_epochs,epoch,decay_epoch).step)
lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_D_B, lr_lambda=LambdaLR(n_epochs,epoch,decay_epoch).step)

NameError: ignored